In [1]:
import vedo
import os
import numpy as np
import napari
from skimage import filters, io

from napari_stress import reconstruction, measurements, sample_data
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

napari.manifest -> 'napari' could not be imported: Could not find file 'builtins.yaml' in module 'napari'


In [2]:
viewer = napari.Viewer()

napari.manifest -> 'napari' could not be imported: Could not find file 'builtins.yaml' in module 'napari'


Make folders:

In [3]:
os.makedirs('./results_sphere', exist_ok=True)
os.makedirs('./results_ellipsoid', exist_ok=True)

for path in ['./results_sphere', './results_ellipsoid']:
    os.makedirs(os.path.join(path, 'napari_stress_analysis_and_napari_stress_pointcloud'), exist_ok=True)
    os.makedirs(os.path.join(path, 'napari_stress_analysis_and_stress_pointcloud'), exist_ok=True)
    os.makedirs(os.path.join(path, 'stress_analysis_and_stress_pointcloud'), exist_ok=True)

## Create synthetic data

Create sphere with padded border

In [6]:
#  for blurring
sigma = 5

In [7]:
radius = 5
padding = 15
sphere = sample_data.make_binary_ellipsoid(major_axis_length=radius, medial_axis_length=radius, minor_axis_length=radius, edge_padding=padding, sampling=0.25)[0]
sphere = filters.gaussian(sphere, sigma=sigma)

Repeat for ellipsoid:

In [8]:
radius_major = 10
radius_minor = 7.5
ellipsoid = sample_data.make_binary_ellipsoid(major_axis_length=radius_major, medial_axis_length=radius_minor, minor_axis_length=radius_minor, edge_padding=padding, sampling=0.25)[0]

## Analyze with napari-stress

### Sphere

In [9]:
dest = os.path.join('./results_sphere/napari_stress_analysis_and_napari_stress_pointcloud/')

Sphere:

In [10]:
results_reconstruction = reconstruction.reconstruct_droplet(sphere,
                                                            voxelsize=np.asarray([1, 1, 1]),
                                                            target_voxelsize=1,
                                                            n_smoothing_iterations=15,
                                                            n_points=256,
                                                            n_tracing_iterations=3,
                                                            resampling_length=1,
                                                            fit_type='fancy',
                                                            edge_type='interior',
                                                            trace_length=20,
                                                            sampling_distance=1,
                                                            remove_outliers=False,
                                                            verbose=False,
                                                            use_dask=False
                                                            )

refined_points = results_reconstruction[3][0]
results_stress_analysis_sphere = measurements.comprehensive_analysis(refined_points,
                                                              max_degree=20,
                                                              n_quadrature_points=590,
                                                              gamma=5,
                                                              verbose=False,
                                                              use_dask=False)

for res in results_reconstruction + results_stress_analysis_sphere:
    if res[2] == 'points':
        viewer.add_points(res[0], **res[1])
    if res[2] == 'vectors':
        viewer.add_vectors(res[0], **res[1])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [11]:
np.savetxt(os.path.join(dest, 'mean_curvatures.csv'), viewer.layers['Result of lebedev quadrature (droplet)'].features['mean_curvature'])
np.savetxt(os.path.join(dest, 'total_stress.csv'), viewer.layers['Result of lebedev quadrature (droplet)'].features['anisotropic_stress'])
np.savetxt(os.path.join(dest, 'lebedev_points.csv'), viewer.layers['Result of lebedev quadrature (droplet)'].data)
np.savetxt(os.path.join(dest, 'expanded_points.csv'), viewer.layers['Result of fit spherical harmonics (deg = 20'].data)
np.savetxt(os.path.join(dest, 'pointcloud.csv'), refined_points)

In [12]:
viewer.close_all()
viewer = napari.Viewer(ndisplay=3)

### Ellipsoid:

In [13]:
dest = os.path.join('./results_ellipsoid/napari_stress_analysis_and_napari_stress_pointcloud/')

In [17]:
results_reconstruction = reconstruction.reconstruct_droplet(ellipsoid,
                                                            voxelsize=np.asarray([1, 1, 1]),
                                                            target_voxelsize=1,
                                                            n_smoothing_iterations=15,
                                                            n_points=256,
                                                            n_tracing_iterations=3,
                                                            resampling_length=1,
                                                            fit_type='fancy',
                                                            edge_type='interior',
                                                            trace_length=20,
                                                            sampling_distance=1,
                                                            remove_outliers=True,
                                                            verbose=False,
                                                            use_dask=False
                                                            )

refined_points = results_reconstruction[3][0]
results_stress_analysis_ellipsoid = measurements.comprehensive_analysis(refined_points,
                                                              max_degree=20,
                                                              n_quadrature_points=590,
                                                              gamma=5,
                                                              verbose=False,
                                                              use_dask=False)


for res in results_reconstruction + results_stress_analysis_ellipsoid:
    if res[2] == 'points':
        viewer.add_points(res[0], **res[1])
    if res[2] == 'vectors':
        viewer.add_vectors(res[0], **res[1])


ValueError: Item wrong length 3303 instead of 3304.

In [ ]:
np.savetxt(os.path.join(dest, 'mean_curvatures.csv'), viewer.layers['Result of lebedev quadrature (droplet)'].features['mean_curvature'])
np.savetxt(os.path.join(dest, 'total_stress.csv'), viewer.layers['Result of lebedev quadrature (droplet)'].features['anisotropic_stress'])
np.savetxt(os.path.join(dest, 'lebedev_points.csv'), viewer.layers['Result of lebedev quadrature (droplet)'].data)
np.savetxt(os.path.join(dest, 'expanded_points.csv'), viewer.layers['Result of fit spherical harmonics (deg = 20'].data)
np.savetxt(os.path.join(dest, 'pointcloud.csv'), refined_points)